In [ ]:
#Imports 
import sys
sys.path.append('/Users/viktoriiaovsianik/Documents/Uni/04_WS2024/06_ADL/Code/ADL-WS-2024')
from scripts.train_cnn import train_model
from scripts.viz import display_images


from datasets.AIArtBench import AIArtbench
from datasets.dataset import Subset
from datasets.preprocessing import CustomDatasetPreprocessor
from torch.utils.data import DataLoader

#### Import dataset

In [4]:
# Define your dataset path and preprocess the data
dataset_path = "/Users/viktoriiaovsianik/Documents/Uni/04_WS2024/06_ADL/Code/ADL-WS-2024/data"
preprocessor = CustomDatasetPreprocessor(dataset_path)
preprocessor.preprocess(fraction=0.01)

# Get dataset splits
train_data, validation_data, test_data = preprocessor.get_splits()

# Prepare the training dataset and DataLoader
train_dataset = AIArtbench(dataframe=train_data, subset=Subset.TRAINING, transform=None)

In [ ]:
#Plot images
train_dataset.plot_images(label_type='human', k=5)
train_dataset.plot_images(label_type='AI', k=5)

# Extract images
#train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# Get unique class names
#classes = train_data['label'].unique()
# Display a few images from the training data
#display_images(data_loader=train_data_loader, classes=classes, num_images=8, save_path="train_images_grid.png")

#### Train baseline

In [ ]:
config = {
    "dataset_path": dataset_path,  # Modify as needed
    "fraction": 0.1,
    "learning_rate": 0.001,
    "scheduler_gamma": 0.9,
    "epochs": 5,
    "batch_size": 64,
    "val_frequency": 3,
    "model_save_dir": "saved_models",
    "debug_mode": "True"
}

train_model(config)